# <center>Enregistrer et charger des modèles </center>

**Source :** https://www.tensorflow.org/tutorials/keras/save_and_load?hl=fr

In [2]:
# Importations
# pip install pyyaml h5py  # Required to save models in HDF5 format
import os
import tensorflow as tf
from tensorflow import keras
print(tf.version.VERSION)

2.11.0


## Jeu de données MNIST

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# 1000 premiers exemples
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


## Construction du modèle 

In [4]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Enregistrer les points de contrôle pendant l'entraînement

Créez un rappel `tf.keras.callbacks.ModelCheckpoint` qui enregistre les pondérations uniquement pendant l'entraînement :

In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
31/32 [============================>.] - ETA: 0s - loss: 1.1249 - sparse_categorical_accuracy: 0.6885
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 2s 28ms/step - loss: 1.1193 - sparse_categorical_accuracy: 0.6910 - val_loss: 0.6928 - val_sparse_categorical_accuracy: 0.7920
Epoch 2/10
31/32 [============================>.] - ETA: 0s - loss: 0.4212 - sparse_categorical_accuracy: 0.8810
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 18ms/step - loss: 0.4189 - sparse_categorical_accuracy: 0.8820 - val_loss: 0.5593 - val_sparse_categorical_accuracy: 0.8250
Epoch 3/10
31/32 [============================>.] - ETA: 0s - loss: 0.2786 - sparse_categorical_accuracy: 0.9274
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 0.2796 - sparse_categorical_accuracy: 0.9270 - val_loss: 0.4758 - val_sparse_categorical_accuracy: 0.8490
Epoch 4/10
29/32 [=======

### Prédiction avec un modèle non-entraîné

In [6]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3248 - sparse_categorical_accuracy: 0.0850 - 490ms/epoch - 15ms/step
Untrained model, accuracy:  8.50%


### Prédiction en chargeant le modèle précédemment entraîné

In [7]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4147 - sparse_categorical_accuracy: 0.8680 - 334ms/epoch - 10ms/step
Restored model, accuracy: 86.80%


## Options de rappel de point de contrôle
Le rappel fournit plusieurs options pour fournir des noms uniques aux points de contrôle et ajuster la fréquence des points de contrôle.

Entraînez un nouveau modèle et enregistrez des points de contrôle nommés de manière unique une fois toutes les cinq époques :

In [8]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [9]:
# récupérer les points de contrôle
os.listdir(checkpoint_dir)

['checkpoint',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0050.ckpt.index']

In [10]:
# point de contrôle le plus récent
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

## Chargement du dernier point de contrôle du modèle 

In [11]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 1s - loss: 0.4883 - sparse_categorical_accuracy: 0.8730 - 1s/epoch - 34ms/step
Restored model, accuracy: 87.30%


## Enregistrer manuellement les poids

In [12]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 1s - loss: 0.4883 - sparse_categorical_accuracy: 0.8730 - 558ms/epoch - 17ms/step
Restored model, accuracy: 87.30%


## Enregistrer le modèle entier

In [13]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 2s 10ms/step - loss: 1.1230 - sparse_categorical_accuracy: 0.6810
Epoch 2/5
32/32 [==============================] - 0s 10ms/step - loss: 0.4308 - sparse_categorical_accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 9ms/step - loss: 0.2772 - sparse_categorical_accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2055 - sparse_categorical_accuracy: 0.9530
Epoch 5/5
32/32 [==============================] - 0s 9ms/step - loss: 0.1531 - sparse_categorical_accuracy: 0.9690


### Rechargez un nouveau modèle Keras à partir du modèle enregistré :

In [14]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 1s - loss: 0.4296 - sparse_categorical_accuracy: 0.8610 - 754ms/epoch - 24ms/step
Restored model, accuracy: 86.10%
32/32 [==============================] - 1s 10ms/step
(1000, 10)


### Format HDF5

In [16]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5


 1/32 [..............................] - ETA: 6:29 - loss: 2.5525 - sparse_categorical_accuracy: 0.0000e+00

 2/32 [>.............................] - ETA: 5:00 - loss: 2.3372 - sparse_categorical_accuracy: 0.1562    

 3/32 [=>............................] - ETA: 4:50 - loss: 2.2555 - sparse_categorical_accuracy: 0.1771

 4/32 [==>...........................] - ETA: 4:40 - loss: 2.2067 - sparse_categorical_accuracy: 0.2031

 5/32 [===>..........................] - ETA: 4:30 - loss: 2.1647 - sparse_categorical_accuracy: 0.2313

 6/32 [====>.........................] - ETA: 4:20 - loss: 2.1847 - sparse_categorical_accuracy: 0.2083

 7/32 [=====>........................] - ETA: 4:10 - loss: 2.0899 - sparse_categorical_accuracy: 0.2723

32/32 [==============================] - 91s 3s/step - loss: 1.1921 - sparse_categorical_accuracy: 0.6630
Epoch 2/5
32/32 [==============================] - 0s 9ms/step - loss: 0.4383 - sparse_categorical_accuracy: 0.8690
Epoch 3/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2871 - sparse_categorical_accuracy: 0.9200
Epoch 4/5
32/32 [==============================] - 0s 9ms/step - loss: 0.2008 - sparse_categorical_accuracy: 0.9520
Epoch 5/5
32/32 [==============================] - 0s 10ms/step - loss: 0.1536 - sparse_categorical_accuracy: 0.9650


In [17]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [18]:
# vérifier son exactitude
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4493 - sparse_categorical_accuracy: 0.8580 - 433ms/epoch - 14ms/step
Restored model, accuracy: 85.80%
